In [58]:
from google.colab import drive
import pandas as pd

In [59]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
train_data = pd.read_csv("/content/drive/My Drive/Deep Learning/fashion-mnist_train.csv")
test_data = pd.read_csv("/content/drive/My Drive/Deep Learning/fashion-mnist_test.csv")

In [61]:
y = train_data['label']
X = train_data.drop(['label'],axis=1)

In [62]:
X_test = test_data.drop(['label'],axis=1)
y_test = test_data['label']

In [63]:
del train_data

In [64]:
del test_data

In [65]:
X = X.values.reshape(-1,28,28,1)

In [66]:
X_test = X_test.values.reshape(-1,28,28,1)

In [67]:
X = X / 255
X_test = X_test / 255

In [68]:
import warnings
warnings.filterwarnings('ignore')

In [69]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPool2D
from keras.utils.np_utils import to_categorical

In [70]:
y = to_categorical(y,num_classes=10)

In [71]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('BWeight.md5',monitor='val_loss',
                            save_best_only=True)

In [72]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))

In [73]:
from keras import optimizers
sgd = optimizers.SGD(lr=0.001, decay=1e-8, momentum=0.9, nesterov=True)

In [74]:
model.compile(optimizer=sgd,loss = 'categorical_crossentropy', metrics=['accuracy'])

In [75]:
from keras.preprocessing.image import ImageDataGenerator

In [76]:
datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        zoom_range = 0.2,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        vertical_flip=False)

In [77]:
datagen.fit(X)



In [78]:
from sklearn.model_selection import train_test_split

In [79]:
X_train, X_val, y_train, y_val = train_test_split(
...     X, y, test_size=0.1)

In [80]:
size_batch = 28

In [81]:
history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=size_batch),
                              epochs = 10,
                              validation_data = (X_val,y_val),
                              verbose = 2,
                              steps_per_epoch = X_train.shape[0] // size_batch,
                              callbacks=[checkpoint],
                             use_multiprocessing=True)

Epoch 1/10
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: BWeight.md5/assets
1928/1928 - 18s - loss: 1.8217 - accuracy: 0.3049 - val_loss: 1.1904 - val_accuracy: 0.6073
Epoch 2/10
INFO:tensorflow:Assets written to: BWeight.md5/assets
1928/1928 - 18s - loss: 1.3338 - accuracy: 0.4933 - val_loss: 0.8432 - val_accuracy: 0.6758
Epoch 3/10
INFO:tensorflow:Assets written to: BWeight.md5/assets
1928/1928 - 18s - loss: 1.1326 - accuracy: 0.5814 - val_loss: 0.7552 - val_accuracy: 0.7068
Epoch 4/10
INFO:tensorflow:Assets written to: BWeight.md5/assets
1928/1928 - 18s - loss: 1.0252 - accuracy: 0.6189 - val_loss: 0.6983 - val_accuracy: 0.7213
Epoch 5/10
INFO:tensorflow:Assets written to: BWeight.md5/assets
1928/1928 - 18s - loss: 0.9544 - accuracy: 0.6455 - val_loss: 0.6871 - va

In [84]:
y_final_preds = model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [85]:
from sklearn.metrics import classification_report

In [86]:
print(classification_report(y_test,y_final_preds))

              precision    recall  f1-score   support

           0       0.73      0.80      0.76      1000
           1       0.95      0.97      0.96      1000
           2       0.48      0.81      0.60      1000
           3       0.74      0.86      0.80      1000
           4       0.68      0.50      0.58      1000
           5       0.92      0.91      0.91      1000
           6       0.35      0.10      0.15      1000
           7       0.88      0.87      0.87      1000
           8       0.96      0.93      0.94      1000
           9       0.90      0.95      0.92      1000

    accuracy                           0.77     10000
   macro avg       0.76      0.77      0.75     10000
weighted avg       0.76      0.77      0.75     10000

